In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from classification import*
from image_processing import*
from utils import*
%matplotlib inline

In [ ]:
# import all data 
import glob
import random
# using mpimg reading png, bins_range = (0, 1)
cars = glob.glob('vehicles/*/*.png')  
notcars = glob.glob('non-vehicles/*/*.png')

print("Number of Cars-Samples:",len(cars), '\nNumber of NotCars-Samples:',len(notcars))

tstcars = get_rand_samples(cars, 6)
tstnotcars = get_rand_samples(notcars, 6)

draw_pics(tstcars, titles=['car']*6)
draw_pics(tstnotcars, titles=['notcar']*6)

In [ ]:
idx = random.randint(0,len(cars)-1)
car = mpimg.imread(cars[idx])
notcar = mpimg.imread(notcars[idx])
print("Size of test image: ", car.shape, 'idx:',idx)

# preprocessing
car *= 255
car = car.astype(np.uint8)
notcar *= 255
notcar = notcar.astype(np.uint8)

In [ ]:
# convert color space
car_ycc = convert_color(car, conv='RGB2YCrCb')
notcar_ycc = convert_color(notcar, conv='RGB2YCrCb')

car_luv = convert_color(car, conv='RGB2LUV')
notcar_luv = convert_color(notcar, conv='RGB2LUV')

car_hls = convert_color(car, conv='RGB2HLS')
notcar_hls = convert_color(notcar, conv='RGB2HLS')

In [ ]:
# Test for spatial bins diagram
spatialcar_rgb = bin_spatial(car)
spatialnotcar_rgb = bin_spatial(notcar)  
spatialcar_ycc = bin_spatial(car_ycc)  
spatialnotcar_ycc = bin_spatial(notcar_ycc)  
spatialcar_luv = bin_spatial(car_luv)  
spatialnotcar_luv = bin_spatial(notcar_luv)  
spatialcar_hls = bin_spatial(car_hls)  
spatialnotcar_hls = bin_spatial(notcar_hls) 

print("Number of spatial bins feature: ", len(spatialcar_rgb))
plot_axes([spatialcar_rgb, spatialcar_ycc, spatialcar_luv, spatialcar_hls], 
          titles= ['spatial bins car(RGB)','spatial bins car(YCrCb)','spatial bins car(LUV)','spatial bins car(HLS)'])
plot_axes([spatialnotcar_rgb, spatialnotcar_ycc, spatialnotcar_luv, spatialnotcar_hls], 
          titles= ['spatial bins notcar(RGB)','spatial bins notcar(YCrCb)',
                   'spatial bins notcar(LUV)','spatial bins notcar(HLS)'])

In [ ]:
# Test for color bins
colorcar_rgb = color_hist(car)
colornotcar_rgb = color_hist(notcar)
colorcar_ycc = color_hist(car_ycc)
colornotcar_ycc = color_hist(notcar_ycc)
colorcar_luv = color_hist(car_luv)
colornotcar_luv = color_hist(notcar_luv)
colorcar_hls = color_hist(car_hls)
colornotcar_hls = color_hist(notcar_hls)

print("Number of hist feature: ", len(colorcar_rgb))
plot_axes([colorcar_rgb, colorcar_ycc, colorcar_luv, colorcar_hls], 
          titles= ["color hist diagram car(RGB)","color hist diagram car(YCC)",
                   "color hist diagram car(LUV)","color hist diagram car(HLS)"])
plot_axes([colornotcar_rgb, colornotcar_ycc, colornotcar_luv, colornotcar_hls], 
          titles= ["color hist diagram notcar(RGB)","color hist diagram notcar(YCC)",
                   "color hist diagram notcar(LUV)","color hist diagram notcar(HLS)"])

In [ ]:
# test for HOG feature
orient = 9
pix_per_cell = 8
cell_per_block = 2
def test_hog(img, orient, pix_per_cell, cell_per_block):
    feat, hog0 = get_hog_features(img[:,:,0], orient, pix_per_cell, cell_per_block, visualise=True)
    feat, hog1 = get_hog_features(img[:,:,1], orient, pix_per_cell, cell_per_block, visualise=True)
    feat, hog2 = get_hog_features(img[:,:,2], orient, pix_per_cell, cell_per_block, visualise=True)
    return hog0, hog1, hog2
    
hogcar_r, hogcar_g, hogcar_b = test_hog(car, orient, pix_per_cell, cell_per_block)
hogcar_y, hogcar_cr, hogcar_cb = test_hog(car_ycc, orient, pix_per_cell, cell_per_block)
hognotcar_r, hognotcar_g, hognotcar_b = test_hog(notcar, orient, pix_per_cell, cell_per_block)
hognotcar_y, hognotcar_cr, hognotcar_cb = test_hog(notcar_ycc, orient, pix_per_cell, cell_per_block)

draw_pics([car, hogcar_r, hogcar_g, hogcar_b, hogcar_y, hogcar_cr, hogcar_cb], 
          cmap=[None]+['gray']*6, titles=['car','R','G','B','Y','Cr','Cb'])
draw_pics([notcar, hognotcar_r, hognotcar_g, hognotcar_b, hognotcar_y, hognotcar_cr, hognotcar_cb], 
          cmap=[None] + ['gray']*6,titles=['notcar','R','G','B','Y','Cr','Cb'])

hogcar_l, hogcar_u, hogcar_v = test_hog(car_luv, orient, pix_per_cell, cell_per_block)
hogcar_h, hogcar_l, hogcar_s = test_hog(car_hls, orient, pix_per_cell, cell_per_block)
hognotcar_l, hognotcar_u, hognotcar_v = test_hog(notcar_luv, orient, pix_per_cell, cell_per_block)
hognotcar_h, hognotcar_l, hognotcar_s = test_hog(notcar_hls, orient, pix_per_cell, cell_per_block)

draw_pics([car, hogcar_l, hogcar_u, hogcar_v, hogcar_h, hogcar_l, hogcar_s], 
          cmap=[None]+['gray']*6, titles=['car','L','U','V','H','L','S'])
draw_pics([notcar, hognotcar_l, hognotcar_u, hognotcar_v, hognotcar_h, hognotcar_l, hognotcar_s], 
          cmap=[None] + ['gray']*6,titles=['notcar','L','U','V','H','L','S'])

In [ ]:
def test_hog_params(channel, orient=None, pix_per_cell=None, cell_per_block=None):
    if orient is None:
        orient_ = [5,6,7,8,9,10]
        pix_per_cell_ = [8]*6
        cell_per_block_ = [2]*6
    if pix_per_cell is None:
        pix_per_cell_ = [5,6,7,8,9,10]
        orient_ = [9] * 6
        cell_per_block_ = [2] * 6
    if cell_per_block is None:
        cell_per_block_ = [1, 2, 3, 4, 5, 6]
        orient_ = [9]*6
        pix_per_cell_=[8]*6
    feats = []
    hogs = []
    titles = []
    for i in range(6):
        feat, hog = get_hog_features(channel, orient_[i], pix_per_cell_[i], cell_per_block_[i], visualise=True)
        feats.append(feat)
        hogs.append(hog)
        if orient is None:
            titles.append('orient={}'.format(i+5))
        if pix_per_cell is None:
            titles.append('pix_per_cell={}'.format(i+5))
        if cell_per_block is None:
            titles.append('cell_per_block={}'.format(i+1))
        
    return feats, hogs, titles

feats, hogs, titles = test_hog_params(car_ycc[:,:,0], pix_per_cell=8, cell_per_block=2)
draw_pics(hogs, cmap=['gray']*6, titles=titles)
feats, hogs_pix, titles_pix = test_hog_params(car_ycc[:,:,0], orient=9, cell_per_block=2)
draw_pics(hogs_pix, cmap=['gray']*6, titles=titles_pix)
feats_cell, hogs_cell, titles_cell = test_hog_params(car_ycc[:,:,0], orient=9, pix_per_cell=8)
plot_axes(feats_cell, titles=titles_cell)

In [ ]:
# Test all features
featcar = single_img_features(car)
featnotcar = single_img_features(notcar)

plot_axes([featcar, featnotcar], titles=['Feature car (not normalized)', 'Feature notcar (not normalized)'])

In [ ]:
# Test for scaling

from sklearn.preprocessing import StandardScaler

# must have many datasets to compare with each other, so that the scaled data can be correct
# singel feature cannot be scaled correctly

X = np.vstack((featcar, featnotcar)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
print('Feature of single dataset: ', scaled_X[0].shape)
plot_axes([scaled_X[0], scaled_X[1]],titles=['Feature car (normalized)', 'Feature notcar (normalized)'])

In [ ]:
# test for jpg format
jpg = mpimg.imread('test.jpg')
jpg = cv2.resize(jpg, (64,64))
spatial_jpg = bin_spatial(jpg)  # size=(32,32)
colorhist_jpg = color_hist(jpg)  # bins_range = (0, 256)
jpg_ycc = convert_color(jpg, conv='RGB2YCrCb')
feat_y_jpg, hogvis_jpg = get_hog_features(jpg_ycc[:,:,0], orient=9, pix_per_cell=8, cell_per_block=2, visualise=True)
feat_jpg = single_img_features(jpg)  # in function automatically converted to ycrcb
draw_pics([jpg, hogvis_jpg], cmap=[None, 'gray'], titles=['JPG image', 'HOG Image'])
plot_axes([spatial_jpg, colorhist_jpg], titles=['Spatial bins', 'Color hist'])
plot_axes([feat_y_jpg, feat_jpg], titles=['HOG features(Y ch)', 'All features(not normalized)'])

In [ ]:
test = False
if test:
    features_cars = extract_features_imglist(cars)
    features_notcars = extract_features_imglist(notcars)
    print('Number of datasets cars: ',len(features_cars))
    print('Number of datasets notcars: ',len(features_notcars))
    print('Feature of single dataset: ', features_cars[0].shape)
else:
    print("No test. Please set 'test' to True")

In [ ]:
train = False
if train:
    svc, scaler, data = train_classifier(cars, notcars)
else:
    print("No Training. Please set 'train' to True")

In [ ]:
overridefile = False

import pickle
import os

filepath='mymodel.p'

if not os.path.isfile(filepath) or overridefile:
    with open(filepath, 'wb') as f:
        pickle.dump((svc,scaler, data), f)
        print("New file saved: ", filepath)
else:
    print('File already exists.')
    print("When you want to override the saved file, please set 'overridefile' to True")

In [ ]:
load = True

if load:
    with open(filepath, 'rb') as f:
        svc, scaler, data = pickle.load(f)
        print('Model loaded.')
else:
    print("Model not loaded. Please set 'load' to True")

In [ ]:
X_train = data[0]
y_train = data[2]
X_test = data[1]
y_test = data[3]
predict_trained_model(svc, X_test, y_test, idx=list(range(0,20)))

In [ ]:
from search_cars import*
import time

raw_img = mpimg.imread('test_images/test4.jpg')
vanish_p = [raw_img.shape[1]//2, int(raw_img.shape[0]/100*58.3)]  # vanishing point

wins_single_layer = gen_windows_single_layer(100, 0, raw_img.shape[1], raw_img.shape[0]//10*7, overlap=0.1)
wins_img_single_layer = draw_boxes(raw_img, wins_single_layer)
wins_all = gen_all_windows(raw_img, vanish_point=vanish_p, overlap=0.5)  
# overlap val should be tuned, 
# overlap too small (close to 0.5) -> not enough windows
# overlap too big (close to 1) ->  too many false windows
wins_img_all = draw_boxes(raw_img, wins_all)

# a full search
overlap = 0.8
n_layers = 5
wins_all = gen_all_windows(raw_img, vanish_point=vanish_p, overlap=overlap, n_layers=n_layers)  
t=time.time()
on_wins = search_windows(raw_img, wins_all, svc, scaler)  # bins_range = (0, 256) because the img is .jpg
t2 = time.time()
print("time used: ", t2-t, "s")
tst_img = draw_boxes(raw_img, on_wins)

draw_pics([wins_img_single_layer, wins_img_all, tst_img], 
          titles=['Windows for one layer (Test)','All windows (Test)','All positive windows'])

In [ ]:
heatmap = np.zeros_like(raw_img[:, :, 0]).astype(np.float)
heatmap = add_heat(heatmap, on_wins)

from scipy.ndimage.measurements import label
threshold = 2  # need to be tuned
heatmap_filtered = apply_threshold(heatmap, threshold)
labels = label(heatmap_filtered)
img_boxes = draw_labeled_bboxes(raw_img, labels)

draw_pics([heatmap, heatmap_filtered, img_boxes], 
          titles=['Heatmap of positive windows','filterted heatmap','end effect'], 
          cmap=['gray', 'gray',None])

In [ ]:
from process_pipeline import *
import time
import glob
imgs = glob.glob('test_images/*.jpg')

overlap = 0.9
threshold = 3
n_layers = 6
w_expand = 3
p = Pipeline(vanish_p, overlap, svc, scaler, threshold, n_layers, w_expand)
imgs_finish = []

for img in imgs:
    raw_img = mpimg.imread(img)
    t = time.time()
    im_finish = p.img_process(raw_img)
    t2 = time.time()
    print("Time to process one single image: ", t2-t, "s", img)
    imgs_finish.append(im_finish)

draw_pics([img for img in imgs_finish[0:3]])
draw_pics([img for img in imgs_finish[3:6]])

In [ ]:
raw_img = mpimg.imread('test_images/test4.jpg')
on_wins = find_cars(raw_img, svc, scaler, cells_per_step=1)
tst_img = draw_boxes(raw_img, on_wins)
heatmap = np.zeros_like(tst_img[:, :, 0]).astype(np.float)
heatmap = add_heat(heatmap, on_wins)

draw_pics([tst_img, heatmap], titles=['All positive windows','Heatmap of positive windows'], cmap=[None, 'gray'])

In [ ]:
threshold = 2
# need to be tuned
heatmap_filtered = apply_threshold(heatmap, threshold)
labels = label(heatmap_filtered)
img_boxes = draw_labeled_bboxes(raw_img, labels)
draw_pics([heatmap_filtered, img_boxes], cmap=['gray', None], titles=['Filtered Heatmap','Image with box(boxes)'])

In [ ]:
from process_pipeline import *
imgs = glob.glob('test_images/*.jpg')

threshold = 2
cells_per_step = 1
window = 64
scale = 1.5
p_subsam = Pipeline_subsamp(svc, scaler, threshold, cells_per_step=cells_per_step, window=window,scale=scale)

imgs_finish = []

for img in imgs:
    raw_img = mpimg.imread(img)
    t = time.time()
    im_finish = p_subsam.img_process(raw_img)
    t2 = time.time()
    print("Time to process one single image: ", t2-t, "s", img)
    imgs_finish.append(im_finish)

draw_pics([img for img in imgs_finish[0:3]])
draw_pics([img for img in imgs_finish[3:6]])

In [ ]:
p_subsam.expLabels=True
raw_img = mpimg.imread('test_images/test1.jpg')
labels = p_subsam.img_process(raw_img)
newlabels = p_subsam.img_process_small(raw_img, labels)
img_boxes = draw_labeled_bboxes(raw_img, newlabels)
draw_pics([raw_img, img_boxes], titles=['raw image', 'with new labels'])

In [ ]:
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
offset = 20
org = Organizer(p_subsam, offset)

output = 'result.mp4'

clip_orig = VideoFileClip("project_video.mp4")

if not os.path.isfile(output):
    res_clip = clip_orig.fl_image(org.img_process) 
    %time res_clip.write_videofile(output, audio=False)
else:
    print('Video Already Exists.')

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>""".format(output))